# Protocol for a distributed synthetic market

This is a protocol for matching **state contingent contracts** that give a payoff linear between an upper and lower bound and at a particular expiry.

<img src="img/BFC.png"  width=500>

For example the contract might pay 1 for each 10mm of rainfall above 30mm. If the amount of rainfall is above 30mm the payoff is 1 and if the outcome is below 30mm the outcome is zero.

These contracts can be combined to construct standard forward and option contracts or event markets (payoffs depending on a discrete set of outcomes). 

<img src="img/StackedBFC.png" width=500>

The protocol uses a chain of signatures to organize buy and sell orders for these contracts. Orders are accepted if they have a valid signature and the trader's positions satisfiy a collateral check across all possible worst outcomes. To maintain an appropriate record of the order book, trades can only be added, and the validity of any trade can be checked by anyone using public keys provided by each participant.

Markets are settled by the market owner adding a new equal upper and lower bound for a market. For example, the lower bound for each market is the highest lower bound for itself of  any sub-market or any previous bound on the same market. Market bounds *telescope*, making a market the *underlying asset* of its sub-markets. 

<img src="img/marketBounds.png" width=600>


## Data structures

### A market

A **market** is defined by:

- `marketRootId`: (integer) market id
- `marketBranchId`: (integer) sub-markets >1 (sub-markets bounded by super-markets)
- `marketMin` - (float) minimum possible outcome
- `marketMax` - (float) maximum possible outcome
- `traderId` - (integer) market owner trader ID
- `previousSig`- (bytes) previous market signature
- `signatureMsg` - (bytes) message for signature
- `signature` - (bytes) signed message
- `timeStampUTC` - (datetime) timestamp
- `timeStampUTCSignature` - (bytes) timestamp signature

E.g.

For an **event market** let:

- Outcome = 0: Event has not happened
- Outcome = 10000: Event has happened

Then if the price of the contract is 5100 this is consistent with a 51% probability that the event happens.


Market with (root = 3, branch = 1); market bounded between (0, 10000); owned by trader 1 and signed with signature 'sig1'.

~~~~

testMarket = {'marketRootId': 3,
 'marketBranchId': 1,
 'marketMin': 0,
 'marketMax': 10000,
 'traderId': 1,
 'previousSig': '<xxx>',
 'signatureMsg': '<xxx>',
 'signature': '<xxx>',
 'timeStampUTC: '<time stamp>',
 'timeStampUTCSignature': '<signature>'}}

~~~~

- Sub markets have `marketMin`/`marketMax` bounded by markets with the same root but lower branch id.
- Any amount of valid markets with the same root/branch can be added and the market will take the highest minimum and the lowest maximum.
- If the minimum is equal to the maximum the market is settled.





### An order

An **order** in a market is defined by:

 - `tradeId`: (integer) trade id
 - `price`:  (float) price of trade
 - `quantity`:  (float) quanitity of trade (positive quantity for bids, negative for offers) 
 - `marketRootId` : (integer) market id
 - `marketBranchId`: (integer) sub-markets > 1 (sub-markets bounded by super-markets)
 - `traderId`:  (integer) trade owner trader ID
 - `previousSig`: (bytes) previous trade signature
 - `signatureMsg` - (bytes) message for signature
 - `signature`: (bytes) signed message
 - `timeStampUTC`: (datetime)  UTC timestamp,
 - `timeStampUTCSignature` : Timestamp signed by time server.
 - `iMatched` - (logical) is trade matched
 - `iRemoved` - (logical) is trade removed
 
E.g. 

Unmatched primary trade (`price = 5000`, `quantity = 10`) on market (`marketRootId = 1`, `marketBranchId = 1`)

~~~~

testTrade = {'traderId': 1,
 'price': 5000,
 'quantity': 10,
 'marketRootId': 3,
 'marketBranchId': 1,
 'signatureMsg': '<xxx>',
 'signature': '<xxx>',
 'timeStampUTC: '<time stamp>',
 'timeStampUTCSignature': '<signature>'}}

~~~~


Trades can be added to the order book but not removed or changed. The flag `iMatched` is set if a match in the same market with an opposite sign and the flag `iRemoved` is set if the matching of an order causes other open orders to breach the maximum allowed collolateral. The `tradeId` is generated automatically.

Trades include a UTC timestamp (`timeStampUTC`) with a signature (`timeStampUTCSignature`) signed by a time server. This establishes the order of trades, trades with with earlier timestamps are not accepted to the chain.




# Methods

Add a new user to the table by storing their public key. This verifies signatures on markets and trades.

## Method: createUser(user)


### Inputs

~~~~

user = {'verifyKey': '<key>'};

~~~~

- `verifyKey` is a unique public key for a user.

### Checks

- `verifyKey` doesn't exist in the user table

### Process

- Add user to table (new `traderId`  = previous `traderId` +1)

## Method: createmarket(market)

Create a new market or alter the bounds on an existing market

### Inputs

~~~~

market = {'marketRootId': 3,
 'marketBranchId': 1,
 'marketMin': 0,
 'marketMax': 10000,
 'traderId': 1,
 'signatureMsg': '<message>',
 'signature': '<signature>'
 'timeStampUTC: '<time stamp>',
 'timeStampUTCSignature': '<signature>'}

~~~~

- `marketRootId`,`marketBranchId` - root and branch number
- `marketMin`,`marketMax` - maximum and minimum possible values
- `traderId` - id of trader who is entitled to change the market
- `previousSig`,`signatureMsg`,`signature` - chain of signatures

### Checks

- If market alread exists, it is owned by the trader
- New market is chained to the correct previous market
* If marketBranchId = 1, market should be chained to the previous root market
* If marketBranchId >1, market  should be chained to the previous market on the same root market
- Signature is correct (`signature` is the correct hash of `signatureMsg` using the `verifyKey` matching to the `traderId`)
- `marketMin` < `marketMax`
- The timestamp is later than current maximum in the order chain.
- The timestamp signature is valid

### Process
- Add new market to the market table
- Construct all possible final combinations of root markets in `outcomeCombinations`

## Method: createTrade()

Add trade to order book and cache after checking trade is valid.

### Inputs

~~~~

trade = {'traderId': 1, 'price': 5000, 'quantity': 1, 'marketRootId': 3,
                'marketBranchId': 1, 'previousSig': '<previous signature>', signatureMsg':
                '<message>', 'signature': '<signature>', 'timeStampUTC': '<time stamp>',
                 'timeStampUTCSignature': '<signature>'}


~~~~

- `traderId`: trader Id
- `price`, `quantity`: price and quantity
- `marketRootId`, `marketBranchId`: market root  and branch (marketBranchId>1 are submarkets) 
- `previousSig`, `signatureMsg`, `signature`: signature chain from previous trade
- `'timeStampUTC'`, `timeStampUTCSignature`: timestamps and signatures


### Checks


### Valid market checks
- `marketRootId` and `marketBranchId` combination exists in in the market table

### Signature and chain checks

- Trade is validly signed (check with public key)
- Trade is validly chained to previously valid trade (furtherst branch on the largest root (tree))
- Trade timestamp is later than most recent in the chain.
- Timstamp signature is valid (using registered public key for time server)
- Collateral checks pass if the trader will have sufficient collateral for this new trade in the worst possible outcome (see **checkCollateral**)

### Process

(if all checks  pass)

- Check if any bids and offers match and set `iMatched = True` for these pairs. Recheck collateral and if it now fails set `iRemoved=True` for open trades until collateral check passes. These removed trades will be ignored for collateral calculation.




## Method: checkCollateral(trade)

Check there is sufficient collateral for trade in the worst possible outcome.

### Inputs

~~~

testTrade = {'traderId': 1,
 'price': 5000,
 'quantity': 10,
 'marketRootId': 3,
 'marketBranchId': 1}


~~~

### Checks

### Process

#### Outcome combinations

- Construct all extreme cominations of root market outcomes where trader is active (**constructOutputCombinations(markets)**)

#### Payoffs 

For each outcome combination:

- Construct payoffs for existing matched trades (**constructPayoffs(orderBook, marketTable)**)
- Construct payoffs for existing open trades
- Construct payoffs for new trade

Net collateral = sum(matchedTradePayoffs) + min(openTradePayoffs, newTradePayoff) 

Collateral check passes if all elements of net collateral > 0






### Matrix representation collateral calculation

The possible set of extreme root market outcomes is a $C \times N$ matrix $\mathbf{M}$ where $C$ is the number of possible market states and $N$ is the number of markets.

At some point in time there are a set of $T$ trades with prices and quantities $\vec{p} = (p_1 , \ldots, p_T)$ and $\vec{q} = (q_1 , \ldots, q_T)$.

The net collateral available to $D$ traders in any one of $C$ market states is:

$\underset{C \times D} {\mathbf{NC} } = (\mathbf{M}^* - \mathbf{P}^*)\mathbf{Q_D}$


Where

$\underset{C \times T}{ \mathbf{M}^*} = \underset{C \times N }{\mathbf{M} } \underset{N \times T}{\mathbf{I}^M}$

$\underset{C \times T}{\mathbf{Q_C}} = \underset{C \times T}{\begin{bmatrix}
           \vec{q} \\
           \vec{q} \\
           \vdots \\
           \vec{q}
         \end{bmatrix}}$
         
$\underset{T \times D}{\mathbf{Q_D}} = \underset{T \times D}{[
           \vec{q}' 
           \vec{q}'
           \ldots 
           \vec{q}'
         ]}$

$\underset{T \times D} {\mathbf{Q_D}^* } = \underset{T \times D } {\mathbf{Q_D}}  \circ \underset{T \times D}{\mathbf{I}^Q }$

$\underset{C \times T}{\mathbf{P}^*} = \underset{C \times T}{\begin{bmatrix}
           \vec{p} \\
           \vec{p} \\
           \vdots \\
           \vec{p}
         \end{bmatrix}}$
         


Two indicator matricies have $I^{M}_{it} = 1$ if trade $t$ is in market $i$ and $I^{Q}_{td} = 1$ if trade $t$ belongs to trader $d$.



$$
\def \vp{
    \begin{bmatrix}
           \vec{p} \\
           \vec{p} \\
           \vdots \\
           \vec{p}
         \end{bmatrix}}
$$

A similar calculation gives the net collateral for each trade in each state:

$\underset{C \times T} {\mathbf{NC}^* } = (\mathbf{M}^* - \mathbf{P}^*) \circ \mathbf{Q_C}$

Where $\circ$ is elementwise mulitplication. This is be convenient for determining worst trade outcome for each state when considering unmatched trades.



#### Example

Number of traders $D$ = 2
Number of markets $N$ = 2 (Outcomes between 0 and 1)

Trades:
- Trader 1: (1, 5000) in market 1
- Trader 1: (2, 4000) in market 2
- Trader 2: (-1, 9000) in market 2

Number of states $C$ = 4

The state of the market is represented by the four possible  extreme outcomes:

$$
\mathbf{M}=
\begin{bmatrix}
    0       & 10000  \\
    10000       & 0  \\
    0      & 0 \\
    10000      & 10000
\end{bmatrix}
$$

The prices and quantities are arranged in a vector:

$$\vec{p} = [5000, 4000 , 9000]$$


$$\vec{q} = [1, 2 ,-1]$$


The market indicator matrix tracks which market each trade belongs to

$$
\mathbf{I^M}=
\begin{bmatrix}
    1       & 0 & 0  \\
    0       & 1 & 1
\end{bmatrix}
$$

The trade indicator matrix tracks which trader each trade belongs to



$$
\mathbf{I^Q}=
\begin{bmatrix}
    1       & 0 \\
    1       & 0 \\
    0       & 1 \\
\end{bmatrix}
$$


Now calculate


$\mathbf{M}^* = \mathbf{M} \mathbf{I}^M =
\begin{bmatrix}
    0       & 10000 & 10000  \\
    10000       & 0 & 0  \\
    0       & 0 & 0  \\
    10000       & 10000 & 10000
\end{bmatrix}
$

$\mathbf{Q_C} = \begin{bmatrix}
    1       & 2 & -1  \\
    1       & 2 & -1  \\
    1       & 2 & -1  \\
    1       & 2 & -1  \\
\end{bmatrix} $

$\mathbf{Q_D}^* = \begin{bmatrix}
    1       & 1   \\
    2       & 2   \\
    -1       & -1  \\
\end{bmatrix} \circ \mathbf{I}^Q  = \begin{bmatrix}
    1       & 0  \\
    2       & 0  \\
    0       & -1 \\
\end{bmatrix} $

$\mathbf{P}^* = \begin{bmatrix}
    5000       & 4000 & 9000  \\
    5000       & 4000 & 9000  \\
    5000       & 4000 & 9000  \\
    5000      & 4000 & 9000  \\
\end{bmatrix} $

The net collateral is


$\mathbf{NC} = (\mathbf{M}^* - \mathbf{P}^*)\mathbf{Q_d}^* = \begin{bmatrix}
    7000       & -1000  \\
    -3000       & 9000 \\
    -13000       & 9000  \\
    17000       & 1000
\end{bmatrix}  $



The columns of the matrix are the collateral outcomes for each of the two traders in the four possible states.

```matlab
M = [0 1; 1 0; 0 0; 1 1]
p = [5000 ,4000, 9000]
q = [1, 2, -1]
IM = [1 0 0;0 1 1] % Convert from marketId [1 2 2]
IQ = [1 0;1 0;0 1] % Convert from traderId [1 1 2]
Mstar = M*IM
Qstar = [q',q'].*IQ
Pstar = [p;p;p;p]
NC = (Mstar - Pstar)*Qstar
```



### Collateral example

In [28]:
import numpy as np
def ind2vec(ind, N=None):
    ind = np.asarray(ind)
    if N is None:
        N = ind.max() + 1
    return (np.arange(N) == ind[:, None]).astype(int)

In [29]:
ind2vec([0,1,1,1,1],2)

array([[1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

In [30]:
# Market outcomes
M = np.array([[0,10000], [10000,0], [0,0], [10000, 10000]])
C, N = M.shape
# prices
p = np.array([5000, 4000, 9000])
# quantity
q = np.array([1, 2, -1])
# Market index
mInd = np.array([1,2,2])
# Trader index
tInd = np.array([1,1,2])
D = tInd.max()


In [15]:
IM = ind2vec(mInd-1, N).T
IM

array([[1, 0, 0],
       [0, 1, 1]])

In [16]:
IQ = ind2vec(tInd-1, D)
IQ

array([[1, 0],
       [1, 0],
       [0, 1]])

In [17]:
Mstar = np.dot(M, IM)
Mstar

array([[    0, 10000, 10000],
       [10000,     0,     0],
       [    0,     0,     0],
       [10000, 10000, 10000]])

In [18]:
QC = np.tile(q, (C,1))
QC

array([[ 1,  2, -1],
       [ 1,  2, -1],
       [ 1,  2, -1],
       [ 1,  2, -1]])

In [19]:
QD = np.tile(q, (D,1)).T
QD

array([[ 1,  1],
       [ 2,  2],
       [-1, -1]])

In [20]:
QDstar = QD * IQ
QDstar

array([[ 1,  0],
       [ 2,  0],
       [ 0, -1]])

In [21]:
Pstar = np.tile(p, (C,1))
Pstar

array([[5000, 4000, 9000],
       [5000, 4000, 9000],
       [5000, 4000, 9000],
       [5000, 4000, 9000]])

In [22]:
NCstar = (Mstar-Pstar)* QC
NCstar

array([[-5000, 12000, -1000],
       [ 5000, -8000,  9000],
       [-5000, -8000,  9000],
       [ 5000, 12000, -1000]])

In [23]:
NC = np.dot((Mstar - Pstar), QDstar)
NC

array([[  7000,  -1000],
       [ -3000,   9000],
       [-13000,   9000],
       [ 17000,  -1000]])

### Colateral condition for matched and unmatched trades

Total collateral is the net collateral for all matched trades plus the worst collateral outcome from unmatched trades. The **total collateral** calculation is:

$\underset{C\times D}{\mathbf{TC}} = \mathbf{NC}_{matched} + \min_D(\mathbf{NC}_{unmatched}^*) = \sum \mathbf{NC}_{matched}^* + \min_D(\mathbf{NC}_{unmatched}^*)  $ 

Where *matched* are trades with exact offsetting trades and each column of $\min_D()$ is the minimum in each state along the rows of $\mathbf{NC}_{unmatched}^*$ consisting only of trader $d$'s trades. 

In the preceeding example no trades are matched so the

In [24]:
# Split out matched and unmatched trades
iMatched = np.array([True, False, False])
NCstar_matched = NCstar[:, np.where(iMatched)].squeeze()
NCstar_matched



array([-5000,  5000, -5000,  5000])

In [25]:
NCstar_unmatched = NCstar[:, np.where( np.logical_not(iMatched))].squeeze()
NCstar_unmatched

array([[12000, -1000],
       [-8000,  9000],
       [-8000,  9000],
       [12000, -1000]])

In [26]:
NCstar_matched.ndim

1

In [27]:
# Stupid python won't sum/min along axis=1 of array
if NCstar_matched.ndim <=1 and NCstar_unmatched.ndim <=1:   
    TC = NCstar_matched + NCstar_unmatched
elif NCstar_matched.ndim <=1 and NCstar_unmatched.ndim >1:
    TC = NCstar_matched + np.min(NCstar_unmatched, axis=1)
elif NCstar_matched.ndim >1 and NCstar_unmatched.ndim <=1:
    TC = np.sum(NCstar_matched, axis=1) + NCstar_unmatched
else:
    TC = np.sum(NCstar_matched, axis=1) + np.min(NCstar_unmatched, axis=1)
    
TC

array([ -6000,  -3000, -13000,   4000])